# AERO Bird's Eye View on drug development

A proof of concept for the bird’s-eye view that uses ClinicalTrials.gov data and the AERO graphing method to analyze all of the registered clinical trials from 10 large pharmaceutical companies over past 20 years.

Enrolment - Participants who have been recruited, screened, and gave their informed consent.

https://www.statnews.com/2019/07/18/clinical-trials-birds-eye-view-drug-development/

In [1]:
import pandas as pd
import matplotlib
import seaborn as sns

%matplotlib inline

In [2]:
df = pd.read_csv('AERO-BirdsEye-Data.csv')
df.head()

,NCT,Sponsor,Title,Summary,Start_Year,Start_Month,Phase,Enrollment,Status,Condition
0,NCT00003305,Sanofi,A Phase II Trial of Aminopterin in Adults and ...,RATIONALE: Drugs used in chemotherapy use diff...,1997,7,Phase 2,75,Completed,Leukemia
1,NCT00003821,Sanofi,Phase II Trial of Aminopterin in Patients With...,RATIONALE: Drugs used in chemotherapy use diff...,1998,1,Phase 2,0,Withdrawn,Endometrial Neoplasms
2,NCT00004025,Sanofi,"Phase I/II Trial of the Safety, Immunogenicity...",RATIONALE: Vaccines made from a person's white...,1999,3,Phase 1/Phase 2,36,Unknown status,Melanoma
3,NCT00005645,Sanofi,Phase II Trial of ILX295501 Administered Orall...,RATIONALE: Drugs used in chemotherapy use diff...,1999,5,Phase 2,0,Withdrawn,Ovarian Neoplasms
4,NCT00008281,Sanofi,"A Multicenter, Open-Label, Randomized, Three-A...",RATIONALE: Drugs used in chemotherapy use diff...,2000,10,Phase 3,0,Unknown status,Colorectal Neoplasms


In [7]:
df[df.Status=='Terminated'].Summary.iloc[5]

'Study to examine the safety and effectiveness of implanted skeletal muscle cells (cells removed from the thigh muscle) into scarred areas of heart muscle after heart attack.'

In [8]:
# Each development has its own unique ID. Number of trials over the past 20 years
df.NCT.nunique()

13748

## How many trials have the 10 companies sponsored over the past 20 years?

In [8]:
df.Sponsor.value_counts()

GSK         2473
Novartis    2320
Pfizer      1970
Merck       1770
Sanofi      1524
JNJ         1143
Roche       1095
Bayer        619
Gilead       417
AbbVie       417
Name: Sponsor, dtype: int64

## Statuses of trials sponsored over the past 20 years

In [4]:
df.Status.value_counts()

Completed                  10568
Terminated                  1285
Recruiting                   800
Active, not recruiting       646
Withdrawn                    291
Not yet recruiting           108
Unknown status                19
Suspended                     16
Enrolling by invitation       15
Name: Status, dtype: int64

In [6]:
df.Phase.value_counts()

Phase 3            4887
Phase 2            3596
Phase 1            2516
Phase 4            2015
Phase 1/Phase 2     322
Phase 2/Phase 3     139
Early Phase 1        10
Name: Phase, dtype: int64

In [7]:
df.isnull().sum()

NCT              0
Sponsor          0
Title          144
Summary          0
Start_Year       0
Start_Month      0
Phase          263
Enrollment       0
Status           0
Condition        0
dtype: int64

In [10]:
df[df.Start_Year==2020].Start_Month.unique()

array([3, 5])

In [3]:
import datetime

df['month_year'] = df[['Start_Month', 'Start_Year']].apply(lambda x: datetime.date(x.Start_Year, x.Start_Month, 1), axis=1)

In [19]:
df.groupby('Sponsor').agg({'Condition': "nunique", "Start_Year": "min", 'NCT': 'count', 'Enrollment': ['sum', 'median', 'mean', 'min', 'max']})

Condition Start_Year   NCT Enrollment                              
           nunique        min count        sum median        mean min    max
Sponsor                                                                     
AbbVie          90       2000   417     109007  141.0  261.407674   0   6000
Bayer          170       1998   619     280250  116.0  452.746365   0  30825
GSK            306       1990  2473    1388740   96.0  561.560857   0  63227
Gilead          75       1999   417     101029  111.0  242.275779   0   6560
JNJ            238       1984  1143     343292  121.0  300.342957   0  15526
Merck          279       1985  1770     946861  154.0  534.949718   0  69274
Novartis       390       1998  2320     865558  125.5  373.085345   0  14075
Pfizer         360       1990  1970     774200  112.0  392.994924   0  84496
Roche          160       1997  1095     399026  122.0  364.407306   0  47208
Sanofi         266       1993  1524     851931  165.5  559.009843   0  31989

In [ ]:
df.groupby(['Condition', 'Sponsor'])

In [46]:
sponsor_study_count = df.groupby('Sponsor')['NCT'].count()

In [34]:
sponsor_status_df = df[(df.Status=='Terminated')|(df.Status=='Completed')].groupby(['Sponsor', 'Status']).agg({'Condition': 'nunique', 'NCT': 'count'})

In [48]:
sponsor_status_pivot = sponsor_status_df.reset_index().pivot(index='Sponsor', columns='Status', values='NCT')
sponsor_status_pivot['terminated_over_completed'] = sponsor_status_pivot['Terminated']/sponsor_status_pivot['Completed'] * 100
sponsor_status_pivot_joined = sponsor_status_pivot.join(sponsor_study_count)
sponsor_status_pivot_joined['completed_ratio'] = sponsor_status_pivot_joined['Completed']/sponsor_status_pivot_joined['NCT']
sponsor_status_pivot_joined.sort_values('terminated_over_completed')

,Completed,Terminated,terminated_over_completed,NCT,completed_ratio
Sponsor,,,,,
GSK,2116,146,6.899811,2473,0.855641
AbbVie,247,18,7.287449,417,0.592326
Bayer,470,44,9.361702,619,0.759289
Roche,772,78,10.103627,1095,0.705023
Sanofi,1232,136,11.038961,1524,0.808399
Novartis,1742,194,11.136625,2320,0.750862
JNJ,868,105,12.096774,1143,0.759405
Merck,1360,221,16.250000,1770,0.768362
Gilead,288,48,16.666667,417,0.690647


In [49]:
df.groupby('Start_Year').agg({'NCT': 'count', 'Condition': 'nunique', 'Sponsor': 'nunique'})

,NCT,Condition,Sponsor
Start_Year,,,
1984,1,1,1
1985,1,1,1
1988,3,2,1
1989,3,1,1
1990,3,3,3
1991,2,2,2
1992,9,6,4
1993,11,9,4
1994,5,3,3


## Diseases with most trials over 20 years

In [53]:
df.groupby('Condition')['NCT'].count().nlargest(n=5)

Condition
Diabetes Mellitus, Type 2                 536
Breast Neoplasms                          388
Pulmonary Disease, Chronic Obstructive    339
Hypertension                              338
Asthma                                    334
Name: NCT, dtype: int64

In [ ]:
## Diseases with the most completed successful Phase 3 trials

In [54]:
p4_completed = df[(df.Phase=='Phase 4') & (df.Status=='Completed')]

In [74]:
p4_completed.groupby('Start_Year').agg({'NCT': "count", "Condition": "nunique"})

,NCT,Condition
Start_Year,,
1992,1,1
1996,1,1
1997,2,2
1998,2,1
1999,6,4
2000,13,11
2001,25,19
2002,80,56
2003,163,77


In [92]:
p4_completed[p4_completed.Start_Year==2004].groupby(['Sponsor'])['Condition'].value_counts().nlargest(n=10)

Sponsor   Condition                             
Novartis  Hypertension                              13
Bayer     Erectile Dysfunction                       7
Novartis  Dermatitis, Atopic                         6
GSK       Asthma                                     5
Merck     Rhinitis, Allergic                         4
GSK       Pulmonary Disease, Chronic Obstructive     3
Merck     Psoriasis                                  3
Pfizer    Urinary Incontinence                       3
Roche     Hepatitis C, Chronic                       3
          Osteoporosis                               3
Name: Condition, dtype: int64

In [60]:
p4_completed.groupby(['Condition', 'Sponsor'])['NCT'].count().nlargest(n=20)

Condition                               Sponsor 
Hypertension                            Novartis    58
Diabetes Mellitus, Type 2               Sanofi      53
Schizophrenia                           JNJ         35
Asthma                                  GSK         33
Influenza, Human                        Sanofi      28
Hepatitis B                             GSK         27
Pulmonary Disease, Chronic Obstructive  GSK         26
Arthritis, Rheumatoid                   Roche       22
Erectile Dysfunction                    Pfizer      18
                                        Bayer       17
Hepatitis                               Roche       17
Diabetes Mellitus                       Sanofi      16
Diabetes Mellitus, Type 2               Novartis    16
Pulmonary Disease, Chronic Obstructive  Novartis    16
Hypercholesterolemia                    Merck       15
Anemia                                  Roche       14
Arthritis, Rheumatoid                   Pfizer      13
Asthma          

In [59]:
p4_completed.groupby('Sponsor').agg({'Condition': 'nunique', 'NCT': 'count'})

,Condition,NCT
Sponsor,,
AbbVie,9,13
Bayer,33,65
GSK,71,241
Gilead,11,24
JNJ,56,128
Merck,67,162
Novartis,105,336
Pfizer,121,273
Roche,29,140


In [72]:
condition_sponsor = df.groupby('Condition')['Sponsor'].unique().reset_index()
condition_sponsor['num_sponsor'] = condition_sponsor['Sponsor'].apply(lambda x: len(x))
condition_sponsor[condition_sponsor.num_sponsor==10].Condition.tolist()

['Arthritis, Rheumatoid',
 'Breast Neoplasms',
 'Carcinoma, Non-Small-Cell Lung',
 'Colitis, Ulcerative',
 'Infection',
 'Kidney Diseases',
 'Leukemia',
 'Liver Diseases',
 'Lung Neoplasms',
 'Lymphoma',
 'Lymphoma, Non-Hodgkin',
 'Multiple Myeloma',
 'Pancreatic Neoplasms',
 'Prostatic Neoplasms',
 'Renal Insufficiency']

In [71]:
condition_1_sponsor = condition_sponsor[condition_sponsor.num_sponsor==1]
condition_1_sponsor['Sponsor'] = condition_1_sponsor['Sponsor'].apply(lambda x: x[0])
condition_1_sponsor.groupby('Sponsor')['Condition'].nunique()

/Users/fionachow/anaconda3/envs/dcuration/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Sponsor
AbbVie       5
Bayer       27
GSK         52
Gilead       3
JNJ         36
Merck       38
Novartis    91
Pfizer      72
Roche       14
Sanofi      52
Name: Condition, dtype: int64

In [51]:
df.groupby(['Condition', 'Phase', "Status"]).agg({'NCT': "count"})

NCT
Condition                          Phase           Status                     
ACTH-Secreting Pituitary Adenoma   Phase 2         Active, not recruiting    2
                                                   Completed                 1
                                                   Not yet recruiting        1
                                   Phase 3         Active, not recruiting    1
                                                   Completed                 3
                                                   Recruiting                1
                                   Phase 4         Completed                 1
Abdominal Abscess                  Phase 3         Completed                 1
                                   Phase 4         Completed                 1
Abdominal Neoplasms                Phase 3         Completed                 1
Abdominal Pain                     Phase 3         Terminated                1
Acne Vulgaris                      Phase 1         Completed                 7
                                   Phase 2         Terminated                1
                                                   Withdrawn                 1
                                   Phase 3         Completed                 8
                                                   Terminated                1
                                   Phase 4         Completed                 7
Acquired Immunodeficiency Syndrome Phase 1         Completed                 7
                                                   Recruiting                1
                                                   Terminated                1
                                   Phase 1/Phase 2 Completed                 1
                                   Phase 2         Completed                 5
                                                   Recruiting                1
                                                   Withdrawn                 1
                                   Phase 2/Phase 3 Completed                 1
                                   Phase 3         Active, not recruiting    1
                                                   Completed                 8
                                   Phase 4         Completed                 2
Acromegaly                         Phase 2         Completed                 3
                                   Phase 3         Completed                 7
...                                                                        ...
Vomiting                           Phase 1         Terminated                1
                                   Phase 2         Completed                 2
                                                   Terminated                1
                                   Phase 3         Completed                 6
                                                   Withdrawn                 1
Von Hippel-Lindau Disease          Phase 2         Completed                 1
Waldenstrom Macroglobulinemia      Phase 2         Completed                 1
Warts                              Phase 1         Completed                 1
                                   Phase 2         Completed                 1
                                   Phase 3         Completed                 1
Weight Loss                        Phase 1         Completed                 1
                                   Phase 2         Completed                 2
                                   Phase 2/Phase 3 Completed                 1
                                   Phase 3         Completed                 2
                                                   Terminated                1
                                   Phase 4         Completed                 1
West Nile Fever                    Phase 2         Completed                 2
Wet Macular Degeneration           Phase 4         Completed                 1
Whooping Cough                     Phase 1    

In [12]:
df.groupby('Sponsor')['Status'].value_counts()

Sponsor   Status                 
AbbVie    Completed                   247
          Recruiting                   67
          Active, not recruiting       59
          Terminated                   18
          Not yet recruiting           14
          Withdrawn                     6
          Enrolling by invitation       5
          Suspended                     1
Bayer     Completed                   470
          Active, not recruiting       56
          Terminated                   44
          Recruiting                   36
          Withdrawn                    10
          Not yet recruiting            3
GSK       Completed                  2116
          Terminated                  146
          Recruiting                   73
          Withdrawn                    72
          Active, not recruiting       42
          Not yet recruiting           12
          Unknown status               10
          Suspended                     2
Gilead    Completed                   288


In [8]:
sponsor_start_year_condition = df.groupby(['Sponsor', 'Start_Year']).agg({'Condition': "nunique"})

In [96]:
df[df.Enrollment > 0].groupby('Condition').agg({'Enrollment': 'sum', 'NCT': 'count'}).nlargest(n=10, columns='Enrollment')

,Enrollment,NCT
Condition,,
"Influenza, Human",347617,318
"Diabetes Mellitus, Type 2",205315,526
Rotavirus Infections,187681,41
Hypertension,162705,330
Breast Neoplasms,162523,372
Malaria,161656,52
Diabetes Mellitus,157546,258
"Pulmonary Disease, Chronic Obstructive",152360,330
Asthma,139490,312


In [9]:
import math

def concat_this(x):
    print(x.tolist())
    return "_".join(x.tolist())

def placeholder_if_na(x, placeholder_value="UNKNOWN"):
    if isinstance(x, float) and math.isnan(x):
        return placeholder_value
    elif isinstance(x, str) and x.lower=='nan':
        return placeholder_value
    return x

In [10]:
df['Phase'] = df['Phase'].apply(lambda x: placeholder_if_na(x))
df['Phase'].isnull().sum()

0

In [11]:
df['phase_status'] = df[['Phase', 'Status']].apply(lambda x: concat_this(x), axis=1)

['Phase 2', 'Completed']
['Phase 2', 'Withdrawn']
['Phase 1/Phase 2', 'Unknown status']
['Phase 2', 'Withdrawn']
['Phase 3', 'Unknown status']
['Phase 1/Phase 2', 'Unknown status']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 2', 'Completed']
['Phase 2', 'Completed']
['Phase 2', 'Completed']
['Phase 2', 'Completed']
['Phase 2', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 2', 'Completed']
['Phase 1/Phase 2', 'Completed']
['Phase 2', 'Completed']
['Phase 3', 'Completed']
['Phase 1/Phase 2', 'Terminated']
['Phase 2', 'Completed']
['Phase 2', 'Terminated']
[

['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Recruiting']
['Phase 3', 'Recruiting']
['Phase 3', 'Active, not recruiting']
['Phase 1', 'Recruiting']
['Phase 1/Phase 2', 'Active, not recruiting']
['Phase 3', 'Active, not recruiting']
['Phase 2', 'Active, not recruiting']
['Phase 4', 'Completed']
['Phase 4', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Recruiting']
['Phase 2', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Recruiting']
['Phase 4', 'Active, not recruiting']
['Phase 2', 'Active, not recruiting']
['Phase 3', 'Completed']
['Phase 2', 'Recruiting']
['Phase 4', 'Completed']
['Phase 1', 'Completed']
['Phase 3', 'Completed']
['Phase 4', 'Completed']
['Phase 2', 'Active, not recruiting']
['Phase 3', 'Active, not recruiting']
['Phase 3', 'Active, not recruiting']
['Phase 1', 'Completed']
['Phase 4', 'Completed']
['Phase 3', 'Recruiting']
['Phase 4', 'Completed']
['Phase 3', 'Completed']
['Phase 4', 'Compl

['Phase 3', 'Completed']
['Phase 4', 'Completed']
['Phase 4', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 3', 'Completed']
['UNKNOWN', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 4', 'Completed']
['Phase 2', 'Completed']
['Phase 3', 'Terminated']
['UNKNOWN', 'Completed']
['Phase 3', 'Completed']
['Phase 1', 'Completed']
['Phase 2', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 4', 'Completed']
['Phase 4', 'Completed']
['Phase 4', 'Completed']
['Phase 2', 'Completed']
['Phase 1', 'Completed']
['Phase 4', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 4', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 4', 'Completed']
['Phase 3', 'Completed']
['Phase 4', 'Completed']
['Phase 4', 'Completed']
['Phase 2', 'Completed']
['Phase 3', 'Completed']
['Phase 4', 'Completed']
['Phase 4', 'Completed']
['Phase 3', 'Terminated']
['Phase 3', 'Terminated

['Phase 4', 'Completed']
['Phase 2', 'Completed']
['Phase 1', 'Completed']
['Phase 1', 'Active, not recruiting']
['Phase 1', 'Terminated']
['Phase 3', 'Withdrawn']
['Phase 1', 'Completed']
['Phase 3', 'Terminated']
['Phase 1', 'Completed']
['Phase 4', 'Completed']
['Phase 2', 'Completed']
['Phase 3', 'Terminated']
['Phase 1', 'Completed']
['Phase 2', 'Terminated']
['Phase 2', 'Terminated']
['Phase 4', 'Completed']
['Phase 3', 'Terminated']
['Phase 1', 'Withdrawn']
['Phase 1', 'Completed']
['Phase 2', 'Completed']
['Phase 1', 'Completed']
['Phase 2', 'Completed']
['Phase 3', 'Terminated']
['Phase 4', 'Recruiting']
['Phase 4', 'Completed']
['Phase 4', 'Terminated']
['Phase 4', 'Completed']
['Phase 1', 'Completed']
['Phase 1', 'Completed']
['Phase 1', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Withdrawn']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 4', 'Completed']
['Phase 1', 'Completed']
['Phase 4', 'Completed']
['Phase 4', 'Completed']
['Phase 1', 'Completed']
['P

['Phase 2', 'Completed']
['Phase 4', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 2', 'Completed']
['Phase 4', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Terminated']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 2/Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 2', 'Completed']
['Phase 4', 'Terminated']
['Phase 1', 'Suspended']
['Phase 4', 'Completed']
['Phase 3', 'Completed']
['Phase 4', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Withdrawn']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 4', 'Co

...

['Phase 1', 'Completed']
['Phase 1', 'Completed']
['Phase 1', 'Completed']
['Phase 2', 'Completed']
['Phase 3', 'Completed']
['UNKNOWN', 'Completed']
['Phase 1', 'Completed']
['UNKNOWN', 'Withdrawn']
['Phase 1', 'Terminated']
['Phase 4', 'Terminated']
['Phase 1', 'Terminated']
['Phase 1', 'Completed']
['Phase 3', 'Completed']
['Phase 1', 'Completed']
['UNKNOWN', 'Completed']
['Phase 2', 'Completed']
['Phase 2', 'Active, not recruiting']
['Phase 1', 'Completed']
['Phase 3', 'Completed']
['Phase 1', 'Terminated']
['Phase 2', 'Terminated']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 1', 'Completed']
['UNKNOWN', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Recruiting']
['Phase 3', 'Completed']
['Phase 1', 'Completed']
['Phase 3', 'Completed']
['Phase 1', 'Completed']
['Phase 4', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Not yet recruiting']
['Phase 1', 'Terminated']
['Phase 1', 'Complete

['Phase 3', 'Active, not recruiting']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Active, not recruiting']
['Phase 3', 'Active, not recruiting']
['Phase 3', 'Active, not recruiting']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 2', 'Active, not recruiting']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Active, not recruiting']
['Phase 3', 'Active, not recruiting']
['Phase 1', 'Completed']
['Phase 2', 'Active, not recruiting']
['Phase 3', 'Recruiting']
['Phase 3', 'Withdrawn']
['Phase 4', 'Recruiting']
['Phase 4', 'Recruiting']
['Phase 3', 'Recruiting']
['Phase 2', 'Recruiting']
['Phase 4', 'Completed']
['Phase 2', 'Recruiting']
['Phase 3', 'Active, not recruiting']
['Phase 2', 'Active, not recruiting']
['Phase 3', 'Active, not recruiting']
['Phase 3', 'Enrolling by invitation']
['Phase 1', 'Recruiting']
['Phase 1/Phase 2', 'Active, not recruiting']
['Phase 1', 'Withdrawn']
['Phase 3', 'Recruiting']
['Phase 2', 'Recruiting']
['Phase 1', '

['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['UNKNOWN', 'Terminated']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 4', 'Completed']
['Phase 4', 'Completed']
['Phase 2', 'Completed']
['Phase 2', 'Completed']
['Phase 2', 'Completed']
['Phase 4', 'Completed']
['Phase 3', 'Completed']
['Phase 4', 'Completed']
['Phase 2', 'Completed']
['Phase 3', 'Completed']
['Phase 4', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Terminated']
['Phase 4', 'Completed']
['Phase 4', 'Completed']
['Phase 2', 'Completed']
['Phase 4', 'Completed']
['Phase 3', 'Completed']
['Phase 4', 'Terminated']
['Phase 3', 'Terminated']
['Phase 3', 'Completed']
['Phase 4', 'Terminated']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 2', 'Completed']
['Phase 1/Phase 2', 'Terminated']
['Phase 1', 'Completed']
['Phase 2', 'Completed']
['Phase 2', 'Completed']
['Phase 3',

['UNKNOWN', 'Withdrawn']
['UNKNOWN', 'Completed']
['Phase 2', 'Completed']
['Phase 4', 'Completed']
['Phase 4', 'Completed']
['Phase 2', 'Completed']
['Phase 2', 'Completed']
['Phase 2', 'Withdrawn']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Terminated']
['Phase 4', 'Completed']
['Phase 4', 'Completed']
['Phase 4', 'Terminated']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 4', 'Completed']
['Phase 2', 'Completed']
['Phase 2', 'Completed']
['Phase 3', 'Completed']
['Phase 4', 'Completed']
['Phase 3', 'Completed']
['Phase 4', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 4', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 1', 'Completed']
['Phase 2', 'Terminated']
['Phase 1', 'Completed']
['Phase 3', 'Terminated']
['Phase 2', 'Completed']
['UNKNOWN', 'Unknown status']
['Phase 2', 'Completed']
['Phase 3', 'Completed']
['Phase 1', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 2', 'Com

['Phase 4', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 3', 'Active, not recruiting']
['Phase 2', 'Terminated']
['Phase 2', 'Completed']
['Phase 2', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Active, not recruiting']
['Phase 4', 'Completed']
['Phase 3', 'Completed']
['Phase 2', 'Completed']
['Phase 1', 'Active, not recruiting']
['Phase 2', 'Terminated']
['Phase 2', 'Completed']
['Phase 2', 'Completed']
['Phase 1/Phase 2', 'Completed']
['Phase 3', 'Completed']
['Phase 1', 'Completed']
['Phase 2', 'Completed']
['Phase 2', 'Active, not recruiting']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Completed']
['Phase 3', 'Active, not recruiting']
['Phase 3', 'Completed']
['Phase 1/Phase 2', 'Active, not recruiting']
['Phase 2', 'Active, not recruiting']
['Phase 3', 'Active, not recruiting']
['Phase 2', 'Terminated']
['Phase 3', 'Completed']
['Phase 3', 'Terminated']
['Phase 2', 'Completed']
[

In [16]:
for condition, groupdf in df.groupby(['Condition', 'Sponsor']):
    print(f"-----{condition} : {groupdf.shape[0]}-----")
    print(groupdf[['Title', 'Enrollment', 'Condition', 'month_year', 'phase_status']].head(2))
    print()
    print(groupdf[['NCT', 'Summary']])
    break

-----('ACTH-Secreting Pituitary Adenoma', 'Novartis') : 10-----
                                                  Title  Enrollment  \
3745  Extension Study to Assess the Safety and Effic...          19   
4090  A Randomized, Double-blind Study to Assess the...         162   

                             Condition  month_year       phase_status  
3745  ACTH-Secreting Pituitary Adenoma  2004-08-01  Phase 2_Completed  
4090  ACTH-Secreting Pituitary Adenoma  2006-12-01  Phase 3_Completed  

              NCT                                            Summary
3745  NCT00171951  Cushing's disease is a rare serious condition ...
4090  NCT00434148  This study will evaluate the safety and effica...
4853  NCT01331239  This exploratory study is a proof of concept s...
4895  NCT01374906  This is a randomized, double-blind, multicente...
5052  NCT01582061  This study provided access to pasireotide sc i...
5283  NCT01915303  This study is to assess whether pasireotide al...
5360  NCT02060383  The

In [17]:
df[df['Status']=='Completed']

,NCT,Sponsor,Title,Summary,Start_Year,Start_Month,Phase,Enrollment,Status,Condition,month_year,phase_status
0,NCT00003305,Sanofi,A Phase II Trial of Aminopterin in Adults and ...,RATIONALE: Drugs used in chemotherapy use diff...,1997,7,Phase 2,75,Completed,Leukemia,1997-07-01,Phase 2_Completed
6,NCT00012389,Sanofi,"A Multicenter, Open-Label, Randomized, Two-Arm...",RATIONALE: Drugs used in chemotherapy use diff...,2000,12,Phase 3,0,Completed,Colorectal Neoplasms,2000-12-01,Phase 3_Completed
7,NCT00017459,Sanofi,The International Tirazone Triple Trial (i3T):...,RATIONALE: Drugs used in chemotherapy use diff...,2000,7,Phase 3,0,Completed,"Carcinoma, Non-Small-Cell Lung",2000-07-01,Phase 3_Completed
8,NCT00021255,Sanofi,Multicenter Phase III Randomized Trial Compari...,Primary objective: - Compare disease-free surv...,2001,4,Phase 3,3222,Completed,Breast Neoplasms,2001-04-01,Phase 3_Completed
9,NCT00025896,Sanofi,"A Prospective Multinational, Multicenter, Clin...",Pompe disease is caused by a deficiency of a c...,2001,5,Phase 2,8,Completed,Glycogen Storage Disease Type II,2001-05-01,Phase 2_Completed
10,NCT00029822,Sanofi,"Long-Term, Efficacy and Safety of Alfuzosin 10...",A study to determine the effect on prevention ...,2001,5,Phase 3,1522,Completed,Urinary Retention,2001-05-01,Phase 3_Completed
11,NCT00029835,Sanofi,"A Randomized, Double-Blind, Placebo-Controlled...",To assess the effect on weight loss and weight...,2001,9,Phase 3,1033,Completed,Dyslipidemias,2001-09-01,Phase 3_Completed
12,NCT00029848,Sanofi,"A Randomized, Double-Blind, Placebo-Controlled...",To assess the effect on weight loss and weight...,2001,10,Phase 3,1045,Completed,Obesity,2001-10-01,Phase 3_Completed
13,NCT00029861,Sanofi,"A Randomized, Double-Blind, Placebo-Controlled...",To assess the effects of weight loss and weigh...,2001,8,Phase 3,3045,Completed,Weight Loss,2001-08-01,Phase 3_Completed
14,NCT00032734,Sanofi,"A Randomized, Double-Blind, Placebo-Controlled...",The study is designed to assess the efficacy o...,2001,6,Phase 2,35,Completed,Hyponatremia,2001-06-01,Phase 2_Completed
